In [1]:
AOI = 'POLYGON ((-85.299088 40.339368, -85.332047 40.241477, -85.134979 40.229427, -85.157639 40.34146, -85.299088 40.339368))'
START_DATE = "2020-06-15"
END_DATE = "2020-08-07"
REQUEST_ID = "11"

In [2]:
import os
import json
import time
import cv2
import rasterio
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio.mask
import shapely

from tqdm.notebook import tqdm
from os.path import join, basename, split
from scipy.ndimage import rotate
from rasterio.features import rasterize, shapes
from rasterio.merge import merge
from shapely.geometry import Polygon, shape, LinearRing
import shapely.wkt
from pathlib import Path
from datetime import datetime

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity, NearestNeighbors
import rasterio.mask as riomask

from code.download.utils import get_tiles, _check_folder, check_nodata, get_min_clouds
from code.download.load_tiles import load_images
from code.index_research import calculate_ndmi

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
default_crs = 'EPSG:4326'

polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(gpd.GeoSeries([polygon]), columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")

In [4]:
NB_USER = os.getenv('NB_USER')
BASE = f"/home/{NB_USER}/work"

API_KEY = os.path.join(BASE, ".secret/sentinel2_google_api_key.json")
sentinel_tiles_path = os.path.join(BASE, "notebooks/pbdnn/sentinel2grid.geojson")
LOAD_DIR = os.path.join(BASE, "satellite_imagery")
RESULTS_DIR = os.path.join(BASE, f"results/dd/moisture_content//{REQUEST_ID}")

BANDS = {'B08', 'B12', 'TCI'}
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 15.0, 'CLOUDY_PIXEL_PERCENTAGE': 10.0, }
PRODUCT_TYPE = 'L2A'

In [5]:
date_tile_info = get_tiles(aoi_filename, sentinel_tiles_path)
loadings = load_images(API_KEY, date_tile_info.tileID.values, START_DATE, END_DATE, LOAD_DIR, BANDS, CONSTRAINTS, PRODUCT_TYPE)
checked = check_nodata(loadings, PRODUCT_TYPE)

try:
    checked = get_min_clouds(checked)
except Exception:
    print(f'No clean raster found for period from {START_DATE} to {END_DATE}, skipping')
    
for i, tile in date_tile_info.iterrows():
    try:
        tile_folder = Path(checked[tile.tileID])
        print(f'filtered: {tile_folder}')
    except Exception as ex:
        print(ex)
        continue

    b08_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B08_10m.jp2' in filename][0]
    b12_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B12_20m.jp2' in filename][0]
    tci_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'TCI_10m.jp2' in filename][0]

filtered: /home/jovyan/work/satellite_imagery/S2B_MSIL2A_20200706T162839_N0214_R083_T16TFK_20200706T205639


In [6]:
ndmi_path = f'{BASE}/notebooks/dd/data/rasters/{REQUEST_ID}_ndmi.tif'
ndmi_path = calculate_ndmi(b08_tile, b12_tile, out_path=ndmi_path)

In [7]:
aoi = gpd.read_file(aoi_filename)

with rasterio.open(tci_tile) as src:
    tci_image, tfs = riomask.mask(
        src, aoi.to_crs(src.crs).geometry, all_touched=False, crop=True)
    
with rasterio.open(ndmi_path) as src:
    ndmi, tfs = riomask.mask(
        src, aoi.to_crs(src.crs).geometry, all_touched=False, crop=True)

In [8]:
with rasterio.open(ndmi_path) as src:
    ndmi_img, tfs = riomask.mask(
        src, aoi.to_crs(src.crs).geometry, all_touched=False, crop=True)
    meta = src.meta
    meta['transform'] = tfs
    meta['width'] = ndmi_img.shape[-1]
    meta['height'] = ndmi_img.shape[-2]

In [9]:
meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': None,
 'width': 1680,
 'height': 1240,
 'count': 1,
 'crs': CRS.from_epsg(32616),
 'transform': Affine(10.0, 0.0, 641870.0,
        0.0, -10.0, 4467290.0)}

In [10]:
class_names = {
    'Bare soil': (-1, -0.7),
    'No canopy cover': (-0.7, -0.4),
    'High water stress': (-0.4, -0.1),
    'Milde water stress': (-0.1, 0.2),
    'Low water stress': (0.2, 0.5),
    'No water stress': (0.5, 1)
}

NUM_CLASSES = len(class_names)
arr = np.array(range(0, NUM_CLASSES)) / NUM_CLASSES

colors = [
    (182, 10, 28),
    (255, 104, 76),
    (227, 152, 2),
    (255, 218, 102),
    (48, 145, 67),
    (138, 206, 126)
]

labels = []

mask = np.zeros((ndmi[0].shape[-2], ndmi[0].shape[-1], 3))
for idx, (name, values) in enumerate(class_names.items()):
    mask[(ndmi[0] >= values[0])&(ndmi[0] <= values[1])] = colors[idx]
    
    labels.append({
        "color": ",".join(list(map(lambda x: str(int(x)), colors[idx]))),
        "name": name
    })

mask[tci_image.transpose(1, 2, 0)==0] = 0
labels = json.dumps(labels)
mask = mask.astype(np.float32)

In [11]:
os.makedirs(RESULTS_DIR, exist_ok=True)
meta.update(
    count=3,
    nodata=0,
    compress='lzw',
    photometric='RGB'
)

result_name = f"{REQUEST_ID}_{START_DATE}_{END_DATE}.tif"
colored_tif = os.path.join(RESULTS_DIR, result_name)

with rasterio.open(colored_tif, 'w', **meta) as dst:
    dst.update_tags(start_date=START_DATE, 
                    end_date=END_DATE, 
                    request_id=REQUEST_ID,
                    name='Moisture content',
                    labels=labels)

    for i in range(mask.shape[-1]):
        dst.write(mask[:,:,i], indexes=i+1)

In [12]:
os.remove(aoi_filename)

In [13]:
json_classes = {i: x for i, x in enumerate(class_names.keys())}

In [14]:
if not os.path.exists('classes_labels.json'):
    with open('classes_labels.json', 'w') as file:
        json.dump(json_classes, file)

In [15]:
remove_aux_rasters = True

if remove_aux_rasters:
    
    try:
        os.remove(ndmi_path)
    except FileNotFoundError:
        print('No helping rasters found')
    
    try:
        os.rmdir(tile_folder)
    except Exception:
        print(f'Error removing directory {tile_folder}')

Error removing directory /home/jovyan/work/satellite_imagery/S2B_MSIL2A_20200706T162839_N0214_R083_T16TFK_20200706T205639
